<h1> Ejercicios Map-Reduce Master Bigdata </h1>

In [1]:
! mkdir -p mrjob/ejercicio1

In [1]:
import os
os.chdir("/media/notebooks/mrjob/mrjob/ejercicio1")

In [3]:
! pwd

/media/notebooks/mrjob/mrjob/ejercicio1


In [4]:
! tail -10 countries.csv

In [5]:
! tail -10 clients.csv

Isis Sorrells  ,regular,US
Chester Abdul  ,regular,ES
Manda Wingate  ,regular,SI
Anna Rappold  ,regular,SB
Albina Lamore  ,malo,SO
Carolyn Machado  ,bueno,ZA
Jeni Espinoza  ,bueno,GS
Charisse Salzman  ,bueno,SS
Dorla Silber  ,bueno,ES
Lilli Bryson  ,malo,LK


In [4]:
%%writefile mrjob-ejercicio1_alt.py
import sys, os, re, csv
from mrjob.job import MRJob, MRStep

class MRJoin(MRJob):

  # Realiza la ordenacion secundaria
  SORT_VALUES = True

  def mapper(self, _, line):
    reader = csv.reader([line], delimiter=',', quotechar='"')
    splits = []
    for r in reader:
        if len(r) == 2:
            splits.append(r[0])
            splits.append(r[1])
        else:
            splits.append(r[0])
            splits.append(r[1])
            splits.append(r[2])

    if len(splits) == 2: # datos de paises
      symbol = 'A' # ordenamos los paises antes que los datos de personas
      country2digit = splits[1]
      yield (country2digit, [symbol, splits[0]])
    else: #  datos de personas
      symbol = 'B'
      country2digit = splits[2]
      if (splits[1] == "bueno"):
          value = 1
          yield (country2digit, [symbol, value])

  def reducer(self, key, values):
    countries = [] # paises primero ya que llevan la clave 'A'
    resultMap = {}
    for value in values:
      if value[0] == 'A':
        countries.append(value)
      if value[0] == 'B':
        for country in countries:
            if country[1] in resultMap:
                resultMap[country[1]] += value[1]
            else:
                resultMap[country[1]] = value[1]
    for key in resultMap:
        yield [key], resultMap[key]
        
if __name__ == '__main__':
    MRJoin.run()

Overwriting mrjob-ejercicio1_alt.py


In [5]:
! python mrjob-ejercicio1_alt.py /media/notebooks/mrjob/mrjob/ejercicio1/countries.csv  /media/notebooks/mrjob/mrjob/ejercicio1/clients.csv > outputlocal

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/mrjob-ejercicio1_alt.root.20180123.162023.510174
Streaming final output from /tmp/mrjob-ejercicio1_alt.root.20180123.162023.510174/output...
Removing temp directory /tmp/mrjob-ejercicio1_alt.root.20180123.162023.510174...


In [6]:
! tail outputlocal

["South Georgia and the South Sandwich Islands"]	1
["Guam"]	3
["Portugal"]	1
["Qatar"]	1
["Canada"]	1
["Somalia"]	1
["South Sudan"]	1
["Turkey"]	1
["United States"]	1
["South Africa"]	1


In [7]:
%%writefile mrjob-ejercicio1.py
import sys, os, re, csv
from mrjob.job import MRJob, MRStep

class MRJoin(MRJob):

  # Realiza la ordenacion secundaria
  SORT_VALUES = True

  def mapper(self, _, line):
    reader = csv.reader([line], delimiter=',', quotechar='"')
    splits = []
    for r in reader:
        if len(r) == 2:
            splits.append(r[0])
            splits.append(r[1])
        else:
            splits.append(r[0])
            splits.append(r[1])
            splits.append(r[2])

    if len(splits) == 2: # datos de paises
      symbol = 'A' # ordenamos los paises antes que los datos de personas
      country2digit = splits[1]
      yield (country2digit, [symbol, splits[0]])
    else: #  datos de personas
      symbol = 'B'
      country2digit = splits[2]
      if (splits[1] == "bueno"):
          value = 1
          yield (country2digit, [symbol, value])

  def reducer_first(self, key, values):
    countries = [] # paises primero ya que llevan la clave 'A'
    for value in values:
      if value[0] == 'A':
        countries.append(value)
      if value[0] == 'B':
        for country in countries:
          yield [country[1]], value[1]
  
  def reducer_second(self, key, values):
    yield (key, sum(values))
        
  def steps(self):
    return [
        MRStep(mapper=self.mapper,
        reducer=self.reducer_first),
        MRStep(reducer=self.reducer_second)
    ]

if __name__ == '__main__':
    MRJoin.run()

Overwriting mrjob-ejercicio1.py


In [8]:
! python mrjob-ejercicio1.py /media/notebooks/mrjob/mrjob/ejercicio1/countries.csv  /media/notebooks/mrjob/mrjob/ejercicio1/clients.csv > outputlocal

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 2...
Creating temp directory /tmp/mrjob-ejercicio1.root.20180123.162209.278196
Running step 2 of 2...
Streaming final output from /tmp/mrjob-ejercicio1.root.20180123.162209.278196/output...
Removing temp directory /tmp/mrjob-ejercicio1.root.20180123.162209.278196...


In [9]:
! tail outputlocal

["Turkey"]	1
["United States"]	1
["South Georgia and the South Sandwich Islands"]	1
["South Sudan"]	1
["Spain"]	3
["Guinea"]	1
["Portugal"]	1
["Qatar"]	1
["Canada"]	1
["Guam"]	3


In [9]:
! hadoop fs -mkdir /tmp/master-map-reduce-ejercicio1

In [10]:
! hadoop fs -put /media/notebooks/mrjob/mrjob/ejercicio1/*.csv  /tmp/master-map-reduce-ejercicio1

In [11]:
! hadoop fs -ls  /tmp/master-map-reduce-ejercicio1

Found 2 items
-rw-r--r--   3 root supergroup       1289 2018-01-15 18:56 /tmp/master-map-reduce-ejercicio1/clients.csv
-rw-r--r--   3 root supergroup       4120 2018-01-15 18:56 /tmp/master-map-reduce-ejercicio1/countries.csv


In [10]:
! hadoop fs -rm -r /tmp/carpeta/ejercicio1-output

Deleted /tmp/carpeta/ejercicio1-output


In [11]:
! python mrjob-ejercicio1.py hdfs:///tmp/master-map-reduce-ejercicio1/* --output-dir hdfs:///tmp/carpeta/ejercicio1-output -r hadoop

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /usr/lib/hadoop/bin...
Found hadoop binary: /usr/lib/hadoop/bin/hadoop
Using Hadoop version 2.6.0
Looking for Hadoop streaming jar in /usr/lib/hadoop...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Creating temp directory /tmp/mrjob-ejercicio1.root.20180123.162232.137253
Copying local files to hdfs:///user/root/tmp/mrjob/mrjob-ejercicio1.root.20180123.162232.137253/files/...
Running step 1 of 2...
  packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.9.0.jar] /tmp/streamjob8989323183883426722.jar tmpDir=null
  Connecting to ResourceManager at yarnmaster/172.18.0.2:8032
  Connecting to ResourceManager at yarnmaster/172.18.0.2:8032
  Total input paths to process : 2
  number of splits:3
  Submitting tokens for job: job_1516719309696_0001
  Submitted appli

In [12]:
! hadoop fs -tail /tmp/carpeta/ejercicio1-output/part-00000

["Canada"]	1
["Guam"]	3
["Guinea"]	1
["Portugal"]	1
["Qatar"]	1
["Somalia"]	1
["South Africa"]	1
["South Georgia and the South Sandwich Islands"]	1
["South Sudan"]	1
["Spain"]	3
["Turkey"]	1
["United States"]	1


In [15]:
%%writefile mrjob-ejercicio2.py
import sys, os, re, csv
from mrjob.job import MRJob, MRStep

class MRJoin(MRJob):

  # Realiza la ordenacion secundaria
  SORT_VALUES = True

  def mapper(self, _, line):
    reader = csv.reader([line], delimiter=',', quotechar='"')
    splits = []
    for r in reader:
        if len(r) == 2:
            splits.append(r[0])
            splits.append(r[1])
        else:
            splits.append(r[0])
            splits.append(r[1])
            splits.append(r[2])

    if len(splits) == 2: # datos de paises
      symbol = 'A' # ordenamos los paises antes que los datos de personas
      country2digit = splits[1]
      yield (country2digit, [symbol, splits[0]])
    else: #  datos de personas
      symbol = 'B'
      country2digit = splits[2]
      if (splits[1] == "bueno"):
          value = 1
          yield (country2digit, [symbol, value])

  def reducer_first(self, key, values):
    countries = [] # paises primero ya que llevan la clave 'A'
    for value in values:
      if value[0] == 'A':
        countries.append(value)
      if value[0] == 'B':
        for country in countries:
          yield [country[1]], value[1]
  
  def reducer_second(self, key, values):
    yield None, [sum(values), key]
 
  def reducer_third(self, _, values):
    yield max(values)
        
  def steps(self):
    return [
        MRStep(mapper=self.mapper,
        reducer=self.reducer_first),
        MRStep(reducer=self.reducer_second),
        MRStep(reducer=self.reducer_third)
    ]

if __name__ == '__main__':
    MRJoin.run()

Overwriting mrjob-ejercicio2.py


In [16]:
! python mrjob-ejercicio2.py /media/notebooks/mrjob/mrjob/ejercicio1/countries.csv  /media/notebooks/mrjob/mrjob/ejercicio1/clients.csv > outputlocal_ejercicio1

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 3...
Creating temp directory /tmp/mrjob-ejercicio2.root.20180123.162432.371155
Running step 2 of 3...
Running step 3 of 3...
Streaming final output from /tmp/mrjob-ejercicio2.root.20180123.162432.371155/output...
Removing temp directory /tmp/mrjob-ejercicio2.root.20180123.162432.371155...


In [17]:
! tail outputlocal_ejercicio1

3	["Spain"]


In [21]:
! hadoop fs -mkdir /tmp/master-map-reduce-ejercicio2

In [22]:
! hadoop fs -put /media/notebooks/mrjob/mrjob/ejercicio1/*.csv  /tmp/master-map-reduce-ejercicio2

In [23]:
! hadoop fs -ls  /tmp/master-map-reduce-ejercicio2

Found 2 items
-rw-r--r--   3 root supergroup       1289 2018-01-15 19:06 /tmp/master-map-reduce-ejercicio2/clients.csv
-rw-r--r--   3 root supergroup       4120 2018-01-15 19:06 /tmp/master-map-reduce-ejercicio2/countries.csv


In [18]:
! hadoop fs -rm -r /tmp/carpeta/ejercicio2-output

Deleted /tmp/carpeta/ejercicio2-output


In [19]:
! python mrjob-ejercicio2.py hdfs:///tmp/master-map-reduce-ejercicio2/* --output-dir hdfs:///tmp/carpeta/ejercicio2-output -r hadoop

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /usr/lib/hadoop/bin...
Found hadoop binary: /usr/lib/hadoop/bin/hadoop
Using Hadoop version 2.6.0
Looking for Hadoop streaming jar in /usr/lib/hadoop...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Creating temp directory /tmp/mrjob-ejercicio2.root.20180123.162458.462251
Copying local files to hdfs:///user/root/tmp/mrjob/mrjob-ejercicio2.root.20180123.162458.462251/files/...
Running step 1 of 3...
  packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.9.0.jar] /tmp/streamjob912531982172073045.jar tmpDir=null
  Connecting to ResourceManager at yarnmaster/172.18.0.2:8032
  Connecting to ResourceManager at yarnmaster/172.18.0.2:8032
  Total input paths to process : 2
  number of splits:3
  Submitting tokens for job: job_1516719309696_0003
  Submitted applic

3	["Spain"]
Removing HDFS temp directory hdfs:///user/root/tmp/mrjob/mrjob-ejercicio2.root.20180123.162458.462251...
Removing temp directory /tmp/mrjob-ejercicio2.root.20180123.162458.462251...


In [20]:
! hadoop fs -tail /tmp/carpeta/ejercicio2-output/part-00000

3	["Spain"]


In [21]:
%%writefile mrjob-ejercicio3.py
import sys, os, re, csv
from mrjob.job import MRJob, MRStep

class MRJoin(MRJob):

  # Realiza la ordenacion secundaria
  SORT_VALUES = True

  def mapper(self, _, line):
    reader = csv.reader([line], delimiter=',', quotechar='"')
    splits = []
    for r in reader:
        if len(r) == 2:
            splits.append(r[0])
            splits.append(r[1])
        else:
            splits.append(r[0])
            splits.append(r[1])
            splits.append(r[2])

    if len(splits) == 2: # datos de paises
      symbol = 'A' # ordenamos los paises antes que los datos de personas
      country2digit = splits[1]
      yield (country2digit, [symbol, splits[0]])
    else: #  datos de personas
      symbol = 'B'
      country2digit = splits[2]
      if (splits[1] == "bueno"):
          value = 1
          yield (country2digit, [symbol, value])

  def reducer_first(self, key, values):
    countries = [] # paises primero ya que llevan la clave 'A'
    for value in values:
      if value[0] == 'A':
        countries.append(value)
      if value[0] == 'B':
        for country in countries:
          yield [country[1]], value[1]
  
  def reducer_second(self, key, values):
    yield None, [sum(values), key]
 
  def reducer_third(self, key, values):
        valuesAgain = []
        maxq = 1
        for value in values:
            if value[0] >= maxq:
                maxq = value[0]
            valuesAgain.append(value)
        for value in valuesAgain:
            if value[0] >= maxq:
                yield maxq, value[1]
        
  def steps(self):
    return [
        MRStep(mapper=self.mapper,
        reducer=self.reducer_first),
        MRStep(reducer=self.reducer_second),
        MRStep(reducer=self.reducer_third)
    ]

if __name__ == '__main__':
    MRJoin.run()

Overwriting mrjob-ejercicio3.py


In [22]:
! python mrjob-ejercicio3.py /media/notebooks/mrjob/mrjob/ejercicio1/countries.csv  /media/notebooks/mrjob/mrjob/ejercicio1/clients.csv > outputlocal_ejercicio3

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 3...
Creating temp directory /tmp/mrjob-ejercicio3.root.20180123.162650.401325
Running step 2 of 3...
Running step 3 of 3...
Streaming final output from /tmp/mrjob-ejercicio3.root.20180123.162650.401325/output...
Removing temp directory /tmp/mrjob-ejercicio3.root.20180123.162650.401325...


In [23]:
! tail outputlocal_ejercicio3

3	["Guam"]
3	["Spain"]


In [121]:
! hadoop fs -mkdir /tmp/master-map-reduce-ejercicio3

In [122]:
! hadoop fs -put /media/notebooks/mrjob/mrjob/ejercicio1/*.csv  /tmp/master-map-reduce-ejercicio3

In [123]:
! hadoop fs -ls  /tmp/master-map-reduce-ejercicio3

Found 2 items
-rw-r--r--   3 root supergroup       1289 2018-01-15 20:18 /tmp/master-map-reduce-ejercicio3/clients.csv
-rw-r--r--   3 root supergroup       4120 2018-01-15 20:18 /tmp/master-map-reduce-ejercicio3/countries.csv


In [24]:
! hadoop fs -rm -r /tmp/carpeta/ejercicio3-output

Deleted /tmp/carpeta/ejercicio3-output


In [25]:
! python mrjob-ejercicio3.py hdfs:///tmp/master-map-reduce-ejercicio3/* --output-dir hdfs:///tmp/carpeta/ejercicio3-output -r hadoop

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /usr/lib/hadoop/bin...
Found hadoop binary: /usr/lib/hadoop/bin/hadoop
Using Hadoop version 2.6.0
Looking for Hadoop streaming jar in /usr/lib/hadoop...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Creating temp directory /tmp/mrjob-ejercicio3.root.20180123.162707.769851
Copying local files to hdfs:///user/root/tmp/mrjob/mrjob-ejercicio3.root.20180123.162707.769851/files/...
Running step 1 of 3...
  packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.9.0.jar] /tmp/streamjob2946271969611816244.jar tmpDir=null
  Connecting to ResourceManager at yarnmaster/172.18.0.2:8032
  Connecting to ResourceManager at yarnmaster/172.18.0.2:8032
  Total input paths to process : 2
  number of splits:3
  Submitting tokens for job: job_1516719309696_0006
  Submitted appli

3	["Guam"]
3	["Spain"]
Removing HDFS temp directory hdfs:///user/root/tmp/mrjob/mrjob-ejercicio3.root.20180123.162707.769851...
Removing temp directory /tmp/mrjob-ejercicio3.root.20180123.162707.769851...


In [26]:
! hadoop fs -tail /tmp/carpeta/ejercicio3-output/part-00000

3	["Guam"]
3	["Spain"]
